# Coletando dados do Twitter usando MongoDB, Pandas e Scikit Learn

In [1]:
#instalando pacote tweepy
!pip install tweepy

In [17]:
#importando modulos tweepy, datetime e json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

In [18]:
# API Twitter == apps.twitter.com
key = "TrDZ0Imv7GCQ3Xl1kor6HO7iu"
skey = "GityD3ntuPF0rmCJKSbPhrqbB6XpQL1X77QTs9jFp6ODqN8NcS"
access_token = "1301510142165483521-wyarruypyS2DrvpRVV06lLw44bijKZ"
access_tokens = "DPpj70azIUwKjS6qk0oNaj0PprkvNxv9xuoNt18K9ZEVV"

In [19]:
# criando as chaves de autenticação
auth = OAuthHandler(key, skey)
auth.set_access_token(access_token, access_tokens)

In [20]:
# criando uma classe para capturar os dados do twitter e armazenar no MongoDB
class MyListener(StreamListener):
  def on_data(self, dados):
    tweet = json.loads(dados)
    created_at = tweet["created_at"]
    id_str = tweet["id_str"]
    text = tweet["text"]
    obj = {"created_at":created_at,"id_str":id_str,"text":text,}
    tweetind = col.insert_one(obj).inserted_id
    print(obj)
    return True

In [21]:
# criando o objeto mylistener
mylistener = MyListener()

In [22]:
# criando o objeto mystream
mystream = Stream(auth, listener = mylistener)

# Preparando a conexão com o MongoDB

In [8]:
#instalando pacote pymongo
!pip install pymongo

In [23]:
# importando a biblioteca pymongo, modulo MongoClient
from pymongo import MongoClient

In [24]:
# criando a conexão com o MongoDB
client = MongoClient('localhost', 27018)

In [25]:
# criando o banco de dados twitterdb
db = client.twitterdb

In [26]:
# criando a collection 'col'
col = db.tweets

In [27]:
# criando lista de palavras chave para buscar nos tweets
keywords = ['Big Data', 'Python', 'Data Mining', 'Data Science']

# Coletando os tweets

In [28]:
# iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)

{'created_at': 'Tue Sep 08 16:27:30 +0000 2020', 'id_str': '1303369380844183552', 'text': 'RT @WebSolu78780638: Check out my Gig on Fiverr: \nhttps://t.co/2v4UmcEf75\n#proggeek #code #python #javascript #Java #programming #Nodejs #e…', '_id': ObjectId('5f57b0f7b5d3ad9d801d53f7')}
{'created_at': 'Tue Sep 08 16:27:33 +0000 2020', 'id_str': '1303369392080728064', 'text': '#يحدث_الآن:\nاليوم الثالث من الدورة الثالثة في معسكر الأمن السيبراني النسائي بعنوان (Cyber Python)،\nيُستعرض فيها كيف… https://t.co/RyX7e8DQbM', '_id': ObjectId('5f57b0fab5d3ad9d801d53f8')}
{'created_at': 'Tue Sep 08 16:27:34 +0000 2020', 'id_str': '1303369396908363777', 'text': 'RT @jayfug: STURGIS: Big new paper out, based on cell-phone data. Takeaways:\n- Strong evidence Sturgis was a super-spreader event\n- Est. li…', '_id': ObjectId('5f57b0fbb5d3ad9d801d53f9')}
{'created_at': 'Tue Sep 08 16:27:35 +0000 2020', 'id_str': '1303369403724115968', 'text': 'RT @RealBossmann: During the first tenure of the @GovernorObaseki 

KeyboardInterrupt: 

In [29]:
# desconectando do twitter
mystream.disconnect()

# Consultando dados no MongoDB

In [30]:
#verificando um documento no collection
col.find_one()

{'_id': ObjectId('5f523b3e72ce23b83d7ed378'),
 'created_at': 'Fri Sep 04 13:03:53 +0000 2020',
 'id_str': '1301868587569872903',
 'text': 'The accuracy.'}

# Análise de Dados com Pandas e Scikit-Learn

In [31]:
# criando um dataset com dados retomados do MongoDB
dataset = [{"created_at": item["created_at"], "text": item["text"],} for item in col.find()]

In [32]:
#importando o módulo Pandas para trabalhar com datasets em Python
import pandas as pd

In [33]:
# Criando um dataframe a partir do dataset
df = pd.DataFrame(dataset)

In [34]:
# imprimindo o dataframe
df

,created_at,text
0,Fri Sep 04 13:03:53 +0000 2020,The accuracy.
1,Fri Sep 04 13:03:53 +0000 2020,RT @josecastillo: Thanks so much to Adafruit (...
2,Fri Sep 04 13:03:55 +0000 2020,RT @riovagas: Python Developer – Rio de Janeir...
3,Fri Sep 04 13:03:56 +0000 2020,RT @Amir__was__here: def func(x): # {\n pri...
4,Fri Sep 04 13:03:56 +0000 2020,I'm using @replit more and more with my pupils...
...,...,...
130,Tue Sep 08 16:28:17 +0000 2020,A biometric surveillance state is not inevitab...
131,Tue Sep 08 16:28:19 +0000 2020,"RT @roccogalatilaw: ""Authorities misinterpreti..."
132,Tue Sep 08 16:28:20 +0000 2020,"RT @Vapingit: Hiya, @SecAzar 👋. Me again.\n\nI..."
133,Tue Sep 08 16:28:20 +0000 2020,RT @mnaotech: Learn Mobile App Development wit...


In [35]:
# importando o modulo scikit learn
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
# Usando o método CountVectorizer para criar uma matriz de documentos
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [38]:
# contando o numero de ocorrencias das principais palavras em nosso dataset
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:20]

,word,count
0,rt,91
1,python,67
2,co,66
3,https,65
4,the,45
5,in,42
6,is,31
7,and,31
8,data,29
9,to,26
